In [9]:
import os
from collections import Counter, defaultdict
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import csv
import pandas as pd
import seaborn as sns

In [2]:
sns.set(style="whitegrid")
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [19]:
ROOT = '/mnt/data0/lucy/manosphere/'
SUBS = ROOT + 'data/submissions/'
SUB_META = ROOT + 'data/subreddits.txt'
LOGS = ROOT + 'logs/'
PEOPLE_FILE = ROOT + 'data/people.csv'
SUB_META = ROOT + 'data/subreddits.txt'

In [11]:
def get_manual_people(): 
    """
    get list of words, add plural forms
    copied from find_people.py
    """
    words = set()
    sing2plural = {}
    with open(PEOPLE_FILE, 'r') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            if row['community'].strip() != 'generic': 
                word_sing = row['word (singular)'].strip()
                plural = row['word (plural)'].strip()
                if word_sing != '':
                    if word_sing.lower() in words: print('REPEAT', word_sing)
                    words.add(word_sing.lower())
                    sing2plural[word_sing] = plural
                if plural != '': 
                    if plural.lower() in words: print('REPEAT', plural)
                    assert word_sing != ''
                    words.add(plural.lower())
    return words, sing2plural

In [20]:
def get_subreddit_categories(): 
    categories = defaultdict(str)
    categories_rev = defaultdict(list)
    with open(SUB_META, 'r') as infile: 
        reader = csv.DictReader(infile)
        for row in reader: 
            name = row['Subreddit'].strip().lower()
            if name.startswith('/r/'): name = name[3:]
            if name.startswith('r/'): name = name[2:]
            if name.endswith('/'): name = name[:-1]
            categories[name] = row['Category after majority agreement']
            categories_rev[row['Category after majority agreement']].append(name)
    return categories, categories_rev

In [21]:
categories, categories_rev = get_subreddit_categories()

In [4]:
with open(LOGS + 'glossword_one_month.temp', 'r') as infile: 
    d = json.load(infile)

In [29]:
people, sing2plural = get_manual_people()
people_counts_sr = defaultdict(Counter)
people_counts_cat = defaultdict(Counter)
for sr in d: 
    cat = categories[sr[8:]]
    for term in d[sr]: 
        if term in sing2plural:
            plural = sing2plural[term]
            if plural not in d[sr]: 
                plural_count = 0
            else: 
                plural_count = d[sr][plural]
            total_count = d[sr][term] + plural_count
            people_counts_sr[sr][term] = total_count
            people_counts_cat[cat][term] += total_count

In [31]:
for sr in people_counts_sr: 
    print(sr)
    for tup in people_counts_sr[sr].most_common(10): 
        if tup[1] > 0: 
            print(tup,)
    print()

2016-03_depression
('gf', 184)
('natural', 163)
('bro', 96)
('void', 95)
('bf', 90)
('tool', 82)
('primary', 61)
('obstacle', 46)
('robot', 45)
('target', 34)

2016-03_foreveralone
('normie', 280)
('gf', 164)
('bro', 90)
('nice guy', 89)
('natural', 64)
('bf', 60)
('alpha', 60)
('jaw', 48)
('beta', 39)
('pua', 34)

2016-03_mensrights
('mra', 1004)
('sjw', 452)
('wing', 379)
('primary', 197)
('mgtow', 182)
('natural', 144)
('target', 139)
('tool', 138)
('misogynist', 135)
('bro', 102)

2016-03_theredpill
('beta', 2651)
('alpha', 2590)
('natural', 659)
('bro', 446)
('mgtow', 388)
('pua', 382)
('tool', 315)
('sjw', 303)
('gf', 296)
('unicorn', 245)

2016-03_thebluepill
('alpha', 695)
('beta', 284)
('mgtow', 176)
('cuck', 136)
('mra', 133)
('bro', 102)
('natural', 99)
('misogynist', 82)
('nice guy', 75)
('alpha male', 75)

2016-03_asktrp
('beta', 1085)
('alpha', 943)
('gf', 310)
('bro', 309)
('natural', 264)
('orbiter', 170)
('nice guy', 139)
('pua', 116)
('bf', 111)
('tool', 100)

2016-03

In [32]:
for cat in people_counts_cat:
    print(cat)
    for tup in people_counts_cat[cat].most_common(10): 
        if tup[1] > 0: 
            print(tup,)
    print()

Health
('natural', 261)
('gf', 242)
('bf', 142)
('bro', 141)
('void', 132)
('tool', 124)
('primary', 83)
('obstacle', 62)
('robot', 56)
('lover', 50)

Incels
('normie', 378)
('gf', 186)
('incel', 135)
('nice guy', 129)
('alpha', 117)
('bro', 104)
('beta', 79)
('natural', 74)
('bf', 66)
('pua', 63)

MRA
('mra', 1035)
('sjw', 463)
('wing', 382)
('primary', 198)
('mgtow', 182)
('tool', 146)
('natural', 145)
('target', 139)
('misogynist', 136)
('bro', 107)

TRP
('beta', 4159)
('alpha', 4002)
('natural', 1032)
('bro', 788)
('gf', 641)
('pua', 530)
('tool', 507)
('mgtow', 466)
('nice guy', 419)
('sjw', 384)

Criticism
('alpha', 731)
('beta', 318)
('mgtow', 218)
('mra', 187)
('cuck', 138)
('bro', 106)
('natural', 106)
('nice guy', 93)
('misogynist', 88)
('sjw', 81)

MGTOW
('mgtow', 2126)
('beta', 251)
('alpha', 179)
('sjw', 152)
('pua', 151)
('bro', 120)
('unicorn', 103)
('gf', 93)
('white knight', 93)
('natural', 92)

PUA
('nice guy', 214)
('pua', 203)
('bro', 178)
('natural', 165)
('alpha',